
# The PET-MAD universal potential

:Authors: Philip Loche [@PicoCentauri](https://github.com/picocentauri),
          Michele Ceriotti [@ceriottm](https://github.com/ceriottm),
          Arslan Mazitov [@abmazitov](https://github.com/abmazitov)

This example demonstrates how to use the PET-MAD model with ASE, [i-PI](https://ipi-code.org) and [LAMMPS](https://lammps.org). PET-MAD is a "universal"
machine-learning forcefield trained on a dataset that aims to incorporate a very high
degree of structural diversity.

The point-edge transformer (PET) is an unconstrained architecture that achieves a high
degree of symmetry compliance through data augmentation during training (see the [PET
paper](https://proceedings.neurips.cc/paper_files/paper/2023/file/fb4a7e3522363907b26a86cc5be627ac-Paper-Conference.pdf)
for more details). The unconstrained nature of the model simplifies its implementation
and structure, making it computationally efficient and very expressive.

The MAD dataset combines "stable" inorganic structures from the
[MC3D dataset](https://mc3d.materialscloud.org/),
2D structures from the
[MC2D dataset](https://mc2d.materialscloud.org/),
and molecular crystals from the
[ShiftML dataset](https://archive.materialscloud.org/record/2022.147)
with "Maximum Atomic Diversity" configurations, generated by distorting the composition
and structure of these stable templates. By doing so, PET-MAD achieves state-of-the-art
accuracy despite the MAD dataset containing fewer than 100k structures. The reference
DFT settings are highly converged, but limited to a PBEsol functional, so the accuracy
against experimental data depends on how good this level of theory is for a given
system. PET-MAD is introduced, and benchmarked for several challenging modeling tasks,
in [this preprint](https://arxiv.org/abs/2503.14118).


Start by importing the required libraries. To use PET-MAD,
and obtain all the necessary dependencies, you can simply use pip
to install the [PET-MAD package](https://github.com/lab-cosmo/pet-mad):

```bash
pip install pet-mad
```


In [ ]:
import os
import subprocess
from copy import copy, deepcopy

# ASE and i-PI scripting utilities
import ase.units
import chemiscope
import matplotlib.pyplot as plt

# pet-mad ASE calculator
import metatomic.torch as mta
import numpy as np
import requests
from ase.optimize import LBFGS
from ipi.utils.mathtools import get_rotation_quadrature_lebedev
from ipi.utils.parsing import read_output, read_trajectory
from ipi.utils.scripting import (
    InteractiveSimulation,
    forcefield_xml,
    motion_nvt_xml,
    simulation_xml,
)
from pet_mad.calculator import PETMADCalculator

## Inference on the MAD test set

We begin by using the ``ase``-compatible calculator to evaluate energy and forces for
a test dataset that contains both hold-out structures from the MAD dataset, and a few
structures from popular datasets ([MPtrj](https://figshare.com/articles/dataset/Materials_Project_Trjectory_MPtrj_Dataset/23713842?file=41619375),
[Alexandria](https://alexandria.icams.rub.de/),
[OC2020](https://paperswithcode.com/dataset/oc20),
[SPICE](https://www.nature.com/articles/s41597-022-01882-6),
[MD22](https://www.science.org/doi/10.1126/sciadv.adf0873)) re-computed
with consistent DFT settings.

### Load the dataset

We fetch the dataset, and load only some of the structures, to speed up the example
runtime on CPU. The model can also run (much faster) on GPUs if you have some at hand.



In [ ]:
filename = "data/mad-test-mad-settings.xyz"
if not os.path.exists(filename):
    url = (
        "https://huggingface.co/lab-cosmo/pet-mad/resolve/"
        "main/benchmarks/mad-test-mad-settings.xyz"
    )
    response = requests.get(url)
    response.raise_for_status()
    with open(filename, "wb") as f:
        f.write(response.content)

test_structures = ase.io.read(filename, "::15")

# also extract reference energetics and metadata
test_energy = []
test_forces = []
test_natoms = []
test_origin = []
subsets = []

for s in test_structures:
    test_energy.append(s.get_potential_energy())
    test_natoms.append(len(s))
    test_forces.append(s.get_forces())
    test_origin.append(s.info["dataset"])
    if s.info["dataset"] not in subsets:
        subsets.append(s.info["dataset"])

test_natoms = np.array(test_natoms)
test_origin = np.array(test_origin)
test_energy = np.array(test_energy)
test_forces = np.array(test_forces, dtype=object)

### Single point energy and forces

PET-MAD is compatible with the metatomic models interface which allows us
to run it with ASE and many other MD engines. For more details see the [metatomic
documentation](https://docs.metatensor.org/metatomic).

We now load the PET-MAD ASE calculator and calculate energy and forces.



In [ ]:
calculator = PETMADCalculator(version="1.0.2", device="cpu")

The model can also be exported in a format that can be used with
external MD engines. This is done by saving the model to a file,
which includes the model architecture and weights.



In [ ]:
calculator.calculator._model.save("pet-mad-v1.0.2.pt")

The model can also be loaded from this torchscript dump, which often
speeds up calculation as it involves compilation, and is functionally
equivalent unless you plan on fine-tuning, or otherwise modifying
the model.



In [ ]:
calculator = mta.ase_calculator.MetatomicCalculator("pet-mad-v1.0.2.pt", device="cpu")

Here, we run the computation on the CPU. If you have a CUDA GPU you can also set
``device="cuda"`` to speed up the computation.



In [ ]:
mad_energy = []
mad_forces = []
mad_structures = []
for structure in test_structures:
    tmp = deepcopy(structure)
    tmp.calc = copy(calculator)  # avoids overwriting results.
    mad_energy.append(tmp.get_potential_energy())
    mad_forces.append(tmp.get_forces())
    mad_structures.append(tmp)

mad_energy = np.array(mad_energy)
mad_forces = np.array(mad_forces, dtype=object)

A parity plot with the model predictions



In [ ]:
tab10 = plt.get_cmap("tab10")
fig, ax = plt.subplots(1, 2, figsize=(6, 3), constrained_layout=True)

ax[0].plot([0, 1], [0, 1], "b:", transform=ax[0].transAxes)
ax[1].plot([0, 1], [0, 1], "b:", transform=ax[1].transAxes)

for i, sub in enumerate(subsets):
    sel = np.where(test_origin == sub)[0]
    ax[0].plot(
        mad_energy[sel] / test_natoms[sel],
        test_energy[sel] / test_natoms[sel],
        ".",
        c=tab10(i),
        label=sub,
    )
    ax[1].plot(
        np.concatenate(mad_forces[sel]).flatten(),
        np.concatenate(test_forces[sel]).flatten(),
        ".",
        c=tab10(i),
    )

ax[0].set_xlabel("MAD energy / eV/atom")
ax[0].set_ylabel("Reference energy / eV/atom")
ax[1].set_xlabel("MAD forces / eV/Å")
ax[1].set_ylabel("Reference forces / eV/Å")

fig.legend(loc="upper center", bbox_to_anchor=(0.55, 1.20), ncol=3)

Explore the dataset using
[chemiscope](http://chemiscope.org)



In [ ]:
chemiscope.show(
    test_structures,
    mode="default",
    properties={
        "origin": test_origin,
        "energy_ref": {
            "target": "structure",
            "values": test_energy / test_natoms,
            "units": "eV/atom",
        },
        "energy_mad": {
            "target": "structure",
            "values": mad_energy / test_natoms,
            "units": "eV/atom",
        },
        "energy_error": {
            "target": "structure",
            "values": np.abs((test_energy - mad_energy) / test_natoms),
            "units": "eV/atom",
        },
        "force_error": {
            "target": "structure",
            "values": [
                np.linalg.norm(f1 - f2) / n
                for (f1, f2, n) in zip(mad_forces, test_forces, test_natoms)
            ],
            "units": "eV/Å",
        },
    },
    shapes={
        "forces_ref": chemiscope.ase_vectors_to_arrows(
            mad_structures, "forces", scale=5
        ),
        "forces_mad": chemiscope.ase_vectors_to_arrows(
            test_structures, "forces", scale=5
        ),
    },
    settings=chemiscope.quick_settings(
        x="energy_mad",
        y="energy_ref",
        symbol="origin",
        structure_settings={"unitCell": True, "shape": ["forces_ref"]},
    ),
)

### How about equivariance‽

The PET architecture does not provide "intrinsically" invariant
energy predictions, but learns symmetry from data augmentation.
Should you worry? The authors of PET-MAD certainly do, and they
have [studied extensively](http://doi.org/10.1088/2632-2153/ad86a0) whether
the symmetry breaking can cause serious artefacts. You can check by
yourself following the procedure below, that evaluates a structures
over a grid of rotations, estimating the variability in energy
(which is around 1meV/atom, much smaller than the test error).



In [ ]:
rotations = get_rotation_quadrature_lebedev(3)

rot_test = test_structures[100]
rot_structures = []
rot_weights = []
rot_energies = []
rot_forces = []
rot_angles = []

for rot, w, angles in rotations:
    tmp = rot_test.copy()
    tmp.positions = tmp.positions @ rot.T
    tmp.cell = tmp.cell @ rot.T
    tmp.calc = copy(calculator)
    rot_weights.append(w)
    rot_energies.append(tmp.get_potential_energy() / len(tmp))
    rot_forces.append(tmp.get_forces())
    rot_structures.append(tmp)
    rot_angles.append(angles)

rot_energies = np.array(rot_energies)
rot_weights = np.array(rot_weights)
rot_angles = np.array(rot_angles)
erot_rms = 1e3 * np.sqrt(
    np.sum(rot_energies**2 * rot_weights) / np.sum(rot_weights)
    - (np.sum(rot_energies * rot_weights) / np.sum(rot_weights)) ** 2
)
erot_max = 1e3 * np.abs(rot_energies.max() - rot_energies.min())
print(f"""
Symmetry breaking, energy:
RMS: {erot_rms:.3f} meV/at.
Max: {erot_max:.3f} meV/at.
""")

You can also inspect the rotational behavior visually



In [ ]:
chemiscope.show(
    rot_structures,
    mode="default",
    properties={
        "delta_energy": {
            "target": "structure",
            "values": 1e3 * (rot_energies - rot_energies.mean()),
            "units": "eV/atom",
        },
        "euler_angles": rot_angles,
    },
    shapes={
        "forces": chemiscope.ase_vectors_to_arrows(
            rot_structures, "forces", scale=50.0
        ),
    },
    settings=chemiscope.quick_settings(
        x="euler_angles[1]",
        y="euler_angles[2]",
        z="euler_angles[3]",
        map_color="delta_energy",
        structure_settings={"unitCell": True, "shape": ["forces"]},
    ),
)

Note also that [i-PI](http://ipi-code.org) provides functionalities to do this
automatically to obtain MD trajectories with a even higher degree of
symmetry-compliance.



## Simulating a complex surface

PET-MAD is designed to be robust and stable when executing sophisticated
modeling workflows. As an example, we consider a slab of an Al-6xxx alloy
(aluminum with a few percent Mg and Si) with some oxygen molecules adsorbed
at the (111) surface.

<div class="alert alert-danger"><h4>Warning</h4><p>The overall Si+Mg concentration in an Al6xxx alloy is far lower than what
  depicted here. This is just a demonstrative example and should not be taken
  as the starting point of a serious study of this system.</p></div>




In [ ]:
al_surface = ase.io.read("data/al6xxx-o2.xyz")

### Geometry optimization with ``ASE``

As a first example, we use the ``ase`` geometry `LBFGS` optimizer to relax the initial
positions. This leads to the rapid decomposition of the oxygen molecules and the
formation of an oxide layer.



In [ ]:
atoms = al_surface.copy()
atoms.calc = calculator

opt = LBFGS(atoms)

traj_atoms = []
traj_energy = []
opt.attach(lambda: traj_atoms.append(atoms.copy()))
opt.attach(lambda: traj_energy.append(atoms.get_potential_energy()))

# stop the optimization early to speed up the example
opt.run(fmax=0.001, steps=20)

Even if the optimization is cut short and far from converged,
the decomposition of the oxygen molecules is apparent, and
leads to a large energetic stabilization



In [ ]:
chemiscope.show(
    structures=traj_atoms,
    properties={
        "index": np.arange(0, len(traj_atoms)),
        "energy": traj_energy,
    },
    mode="default",
    settings=chemiscope.quick_settings(trajectory=True),
)

### Molecular dynamics with atoms exchange with ``i-PI``

The geometry optimization shows the high reactivity of
this surface, but does not properly account for finite
temperature and does not sample the diffusion of solute
atoms in the alloy (which is mediated by vacancies).

We use [i-PI](http://ipi-code.org) to perform a
molecular dynamics trajectory at 800K, combined with
Monte Carlo steps that swap the nature of atoms, allowing the simulation
to reach equilibrium in the solute-atoms distributions
without having to introduce vacancies or wait for the
very long time scale needed for diffusion.



The behavior of i-PI is controlled by an XML input file.
The ``utils.scripting`` module contains several helper
functions to generate the basic components.

Here we use a ``<motion mode="multi">`` block to combine
a MD run with a ``<motion mode="atomswap">`` block that
attempts swapping atoms, with a Monte Carlo acceptance.



In [ ]:
motion_xml = f"""
<motion mode="multi">
    {motion_nvt_xml(timestep=5.0 * ase.units.fs)}
    <motion mode="atomswap">
        <atomswap>
            <nxc> 0.1 </nxc>
            <names> [ Al, Si, Mg, O]  </names>
        </atomswap>
    </motion>
</motion>
"""

input_xml = simulation_xml(
    structures=[al_surface],
    forcefield=forcefield_xml(
        name="pet-mad",
        mode="direct",
        pes="metatomic",
        parameters={"model": "pet-mad-v1.0.2.pt", "template": "data/al6xxx-o2.xyz"},
    ),
    motion=motion_xml,
    temperature=800,
    verbosity="high",
    prefix="nvt_atomxc",
)

print(input_xml)

The simulation can be run from a Python script or the command line. By changing the
forcefield interface from ``direct`` to the use of a socket, it is also possible to
execute separately ``i-PI`` and the ``metatomic`` driver.



In [ ]:
sim = InteractiveSimulation(input_xml)
sim.run(80)

The simulation generates output files that can be parsed and visualized from Python.



In [ ]:
data, info = read_output("nvt_atomxc.out")
trj = read_trajectory("nvt_atomxc.pos_0.xyz")

fig, ax = plt.subplots(1, 1, figsize=(4, 3), constrained_layout=True)

ax.plot(data["time"], data["potential"], "b-", label="potential")
ax.plot(data["time"], data["conserved"] - 4, "k-", label="conserved")
ax.set_xlabel("t / ps")
ax.set_ylabel("energy / ev")
ax.legend()

The trajectory (which is started from oxygen molecules placed on top of the surface)
shows quick relaxation to an oxide layer. If you look carefully, you'll also see that
Mg and Si atoms tend to cluster together, and accumulate at the surface.



In [ ]:
chemiscope.show(
    structures=trj,
    properties={
        "time": data["time"][::10],
        "potential": data["potential"][::10],
        "temperature": data["temperature"][::10],
    },
    mode="default",
    settings=chemiscope.quick_settings(
        map_settings={
            "x": {"property": "time", "scale": "linear"},
            "y": {"property": "potential", "scale": "linear"},
        },
        structure_settings={
            "unitCell": True,
        },
        trajectory=True,
    ),
)

### Molecular dynamics with ``LAMMPS``

We now run the same MD with [LAMMPS](https://lammps.org). To run a LAMMPS
calculation with a ``metatomic`` potential, one needs a LAMMPS build that contains an
appropriate pair style. You can compile it from [source](https://github.com/metatensor/lammps), or fetch it from the `metatensor` channel on
conda. One can then just include in the input a ``pair_style metatomic`` that points
to the exported model and a single ``pair_coeff`` command that specifies the mapping
from LAMMPS types to the atomic types the model can handle. The first two arguments
must be ``* *`` so as to span all LAMMPS atom types. This is followed by a list of N
arguments that specify the mapping of metatomic atomic types to LAMMPS types, where N
is the number of LAMMPS atom types.



In [ ]:
with open("data/al6xxx-o2.in", "r") as f:
    print(f.read())

<div class="alert alert-danger"><h4>Warning</h4><p>Be aware that the extensions are compiled files and depend on your operating system.
  Usually you have re-export the extensions for different systems! You can do this
  by running the appropriate parts of this file, or using the ``mtt export``
  command-line utility.</p></div>

We also save the geometry to a LAMMPS data file and finally run the simulation.



In [ ]:
ase.io.write("al6xxx-o2.data", al_surface, format="lammps-data", masses=True)

subprocess.check_call(["lmp", "-in", "data/al6xxx-o2.in"])

The resulting trajectory is qualitatively consistent with what
we observed with ``i-PI``.



In [ ]:
lmp_trj = ase.io.read("trajectory.xyz", ":")

chemiscope.show(structures=lmp_trj, mode="structure")